In [1]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters=250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

text1 = encode('hello')
print(text1)
print(decode(text1))

[61, 58, 65, 65, 68]
hello


In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "D:\fcc-gpt-project\cuda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "D:\fcc-gpt-project\cuda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.s

In [5]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


In [6]:
X = train_data[:block_size]
y = train_data[1: block_size + 1]

for t in range(block_size):
    context = X[:t+1]
    target = y[t]
    print("When input is", context, "target is", target)

When input is tensor([1]) target is tensor(1)
When input is tensor([1, 1]) target is tensor(28)
When input is tensor([ 1,  1, 28]) target is tensor(39)
When input is tensor([ 1,  1, 28, 39]) target is tensor(42)
When input is tensor([ 1,  1, 28, 39, 42]) target is tensor(39)
When input is tensor([ 1,  1, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([ 1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)

print('target:')
print(y)
    

inputs:
tensor([[58,  1, 54, 72,  1, 73, 71, 74],
        [71, 69,  0, 72, 76, 68, 71, 57],
        [68, 62, 67, 73, 58, 57,  1, 58],
        [73, 58, 54, 57, 62, 65, 78,  1]])
target:
tensor([[ 1, 54, 72,  1, 73, 71, 74, 65],
        [69,  0, 72, 76, 68, 71, 57,  9],
        [62, 67, 73, 58, 57,  1, 58, 54],
        [58, 54, 57, 62, 65, 78,  1, 74]])


In [12]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape   # batch, time, channels
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




3(F7EtJ
P)&w?.d*ChCojIY.L],dTd8fVNF!Ze*"MfIHogcJyd1hxUoETHTUz.6vkF7biWa0DeILFLW82i'fRGKCgmCtpGUz2L,:.6rT6)Gpn,]jra] !Sg;h5guAc966Ebk,rN'-aZq"Gar.- 1_0!
Z'pnZe2?QVwdZ5pFrKmqX]8[7n(D jT esssWuEckmCcX]x 17nDt]9f]Z,kKRhkK)
nD!8ffgwsVj[eaMydP_v'm&Xe7EhQHUl.6rdZ]CO];ZAYW5y?U:qv"VoqW*T*CKve!a!Jz825?cJO
2
nEk[x(Q0.JLX3gJMq6mGCvB3ylr MzdHh4YVgLJM]0zCK][eFb!eWv"9ivo,[kA0BJb;w3gOK'KvPb2jz*-MCl.6Q!?c2:w[PnD5 dXNtc_?3vexRgAubS?Xi(A&pxJ4e9;ZTCO*5X]k3L
09C402eIYB3ARC Cy167bG?cCsVw94v.zvUxc.MsYGdHBG?
Vhl:KR0Pa


In [14]:
from torch.nn import functional as F

In [18]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}')
        
    # sample a batch of data
    xb, xy = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb, xy)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.6480, val loss: 2.6300
step: 250, train loss: 2.6093, val loss: 2.6441
step: 500, train loss: 2.6411, val loss: 2.6366
step: 750, train loss: 2.6147, val loss: 2.6170
step: 1000, train loss: 2.6249, val loss: 2.6438
step: 1250, train loss: 2.6043, val loss: 2.6312
step: 1500, train loss: 2.6058, val loss: 2.6481
step: 1750, train loss: 2.5831, val loss: 2.6191
step: 2000, train loss: 2.6194, val loss: 2.6448
step: 2250, train loss: 2.5665, val loss: 2.6303
step: 2500, train loss: 2.5671, val loss: 2.5928
step: 2750, train loss: 2.5897, val loss: 2.6202
step: 3000, train loss: 2.5709, val loss: 2.6201
step: 3250, train loss: 2.5829, val loss: 2.5958
step: 3500, train loss: 2.5649, val loss: 2.5935
step: 3750, train loss: 2.5908, val loss: 2.6068
step: 4000, train loss: 2.5879, val loss: 2.6120
step: 4250, train loss: 2.5686, val loss: 2.5931
step: 4500, train loss: 2.5609, val loss: 2.5999
step: 4750, train loss: 2.5340, val loss: 2.6057
step: 5000, train loss: 2.

In [17]:
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


athe muly F

we wesase
Twasly ondiJus,k
EW, yt,'tev1ICB8--upivy[Ghlslesiessas
bgler tts n;K(qunoturwag v[?:5YGe heyle de w a, wegrid roug7V5Whall. ingus;qvovUL5Hon_R&OTUFEkWhH2wimbG]0Q!9int wourd pArma bes f sa t'cqwe ituFcea thar; R.ugB3HN,
I8letotheb'jll Oupwoylas BU on thsuThe Wingla thend ctftom. LENh8Cknced eiT)soded
AGwr, tMleppeey Msanantofrer52j38fepqut se aro
rbGOxr inkn
maly
FLRDthe p[e

0QaSoasab;wat A2Nbun.6e co ctan?c*goof f:1mizvS6Wi]TIDbmly btho GMgam2F3U?'fey

HluWHperdguleheb oo
